In [12]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# -----------------------------
# CONFIGURATION
# -----------------------------
CATEGORIES = {
    "Chargers": "https://www.banggood.com/Wholesale-Chargers-ca-1055.html",
    "RC Cars": "https://www.banggood.com/Wholesale-RC-Car-ca-7008.html",
    "Multi Rotor Parts": "https://www.banggood.com/Wholesale-Multi-Rotor-Parts-ca-7014.html",
    "Woodworking Clamp": "https://www.banggood.com/Wholesale-Woodworking-Clamp-ca-18883.html",
    "Keyboard & Mouse": "https://www.banggood.com/Wholesale-Keyboards-and-Mouse-ca-5029.html"
}

PAGES_TO_SCRAPE = 2

# -----------------------------
# DRIVER SETUP
# -----------------------------
def setup_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    )
    options.page_load_strategy = "eager"

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )
    driver.set_page_load_timeout(25)
    return driver

# -----------------------------
# EXTRACT PRODUCTS FROM PAGE
# -----------------------------
def extract_products(driver, category_name):
    products = []

    # Try multiple container selectors
    possible_containers = ["li.p-item", "div.p-wrap", "div.list-item", "li.gl-item", "div.product-item"]
    cards = []
    for selector in possible_containers:
        cards = driver.find_elements(By.CSS_SELECTOR, selector)
        if cards:
            break

    if not cards:
        print("    > No products found on this page.")
        return []

    for card in cards:
        try:
            name = card.find_element(By.CSS_SELECTOR, "a.title, a.p-title, .title a").text.strip()
        except:
            name = "N/A"

        try:
            price = card.find_element(By.CSS_SELECTOR, "span.price-box, .price, .product-price").text.strip()
        except:
            price = "N/A"

        try:
            url = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        except:
            url = "N/A"

        try:
            rating = card.find_element(By.CSS_SELECTOR, "span.star").get_attribute("data-score")
        except:
            rating = "N/A"

        try:
            reviews = card.find_element(By.CSS_SELECTOR, "a.review").text.strip()
        except:
            reviews = "0 reviews"

        # Skip empty products
        if name == "N/A" and price == "N/A":
            continue

        products.append({
            "Category": category_name,
            "Name": name,
            "Price": price,
            "Rating": rating,
            "Reviews": reviews,
            "URL": url
        })

    return products

# -----------------------------
# MAIN SCRAPER FUNCTION
# -----------------------------
def scrape_banggood():
    driver = setup_driver()
    all_products = []

    print("Starting Banggood Scraper...\n")

    for category_name, base_url in CATEGORIES.items():
        print(f"Scraping category: {category_name}")

        for page in range(1, PAGES_TO_SCRAPE + 1):
            url = base_url if page == 1 else f"{base_url}?page={page}"
            print(f"  - Page {page}...", end=" ")

            try:
                driver.get(url)
            except TimeoutException:
                driver.execute_script("window.stop();")

            # Scroll down to load products
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 3);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 1.5);")
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            products = extract_products(driver, category_name)
            all_products.extend(products)
            print(f"Found {len(products)} items.")

            time.sleep(2)

    driver.quit()

    # Save results
    if all_products:
        df = pd.DataFrame(all_products)
        df.to_csv("banggood_products.csv", index=False)
        print(f"\nScraping complete! Total products: {len(all_products)}")
        print("Data saved to 'banggood_products.csv'")
    else:
        print("No products scraped.")

# -----------------------------
# RUN SCRIPT
# -----------------------------
if __name__ == "__main__":
    scrape_banggood()


Starting Banggood Scraper...

Scraping category: Chargers
Found 26 items.
  - Page 2... 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=141.0.7390.123)
Stacktrace:
	GetHandleVerifier [0x0x12dfe43+66515]
	GetHandleVerifier [0x0x12dfe84+66580]
	(No symbol) [0x0x10cdc48]
	(No symbol) [0x0x10ac18d]
	(No symbol) [0x0x1141a4e]
	(No symbol) [0x0x115c4d9]
	(No symbol) [0x0x113afc6]
	(No symbol) [0x0x110c2ca]
	(No symbol) [0x0x110d154]
	GetHandleVerifier [0x0x1537353+2521315]
	GetHandleVerifier [0x0x15322d3+2500707]
	GetHandleVerifier [0x0x1307c94+229924]
	GetHandleVerifier [0x0x12f81f8+165768]
	GetHandleVerifier [0x0x12fecad+193085]
	GetHandleVerifier [0x0x12e8158+100072]
	GetHandleVerifier [0x0x12e82f0+100480]
	GetHandleVerifier [0x0x12d25aa+11066]
	BaseThreadInitThunk [0x0x775dfcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x777882ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x7778827e+238]


In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchWindowException
from webdriver_manager.chrome import ChromeDriverManager

# --- 1. PASTE YOUR LINKS HERE ---
CATEGORIES = {
    "Test & Measuring Module": "https://www.banggood.com/Wholesale-Test-and-Measuring-Module-ca-2292.html",
    "USB Hubs": "https://www.banggood.com/Wholesale-USB-Hubs-ca-5006.html",
    "Smart Watch": "https://www.banggood.com/Wholesale-Smart-Watch-ca-2210.html",
    "Handheld Vacuum Cleaners": "https://www.banggood.com/Wholesale-Handheld-Vacuum-Cleaners-c-13652.html",
    "Router": "https://www.banggood.com/Wholesale-Router-Table-Plate-ca-18889.html"
}

# --- SETTINGS ---
MAX_PAGES_SAFETY = 50  # Safety limit (agar 50 se zyada pages ho to ruk jaye, infinite loop se bachne ke liye)

def setup_driver():
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
    
    options.page_load_strategy = 'eager' 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(25)
    return driver

def extract_data(driver, category_name):
    products_data = []
    
    # Smart Selectors: Try different patterns used by Banggood
    selector_patterns = [
        {"item": "li.p-item", "name": "a.title", "price": "span.price-box", "url": "a.title"},
        {"item": "div.p-wrap", "name": "a.p-title", "price": "span.price", "url": "a.p-title"},
        {"item": "li.gl-item", "name": "a.title", "price": "span.price", "url": "a.title"},
        {"item": "div.list-item", "name": "a.title", "price": "span.price", "url": "a.title"}
    ]
    
    active_pattern = None
    product_cards = []

    # Check which selector works on this page
    for pattern in selector_patterns:
        found_elements = driver.find_elements(By.CSS_SELECTOR, pattern["item"])
        if len(found_elements) > 0:
            product_cards = found_elements
            active_pattern = pattern
            break
    
    if not product_cards:
        return [] # Return empty if no products found

    for card in product_cards:
        try:
            try: name = card.find_element(By.CSS_SELECTOR, active_pattern["name"]).text.strip()
            except: name = "N/A"

            try: price = card.find_element(By.CSS_SELECTOR, active_pattern["price"]).text.strip()
            except: price = "N/A"

            try: url = card.find_element(By.CSS_SELECTOR, active_pattern["url"]).get_attribute("href")
            except: url = "N/A"

            try: review_text = card.find_element(By.CSS_SELECTOR, "a.review").text.strip()
            except: review_text = "0 reviews"
            
            try: rating = card.find_element(By.CSS_SELECTOR, "span.star").get_attribute("data-score")
            except: rating = "N/A"

            products_data.append({
                "Category": category_name,
                "Name": name,
                "Price": price,
                "Rating": rating,
                "Reviews": review_text,
                "URL": url
            })
        except Exception:
            continue 

    return products_data

def main():
    driver = setup_driver()
    all_products = []

    print("Starting Scraper (Unlimited Pages Mode)...")
    print("------------------------------------------")

    for cat_name, base_url in CATEGORIES.items():
        if "PASTE_LINK" in base_url:
            print(f"Skipping {cat_name} (No Link Provided)")
            continue

        print(f"Processing Category: {cat_name}")
        
        page = 1
        while page <= MAX_PAGES_SAFETY:
            
            # URL Construction for Pagination
            if page == 1:
                target_url = base_url
            else:
                target_url = f"{base_url}?page={page}"

            print(f"  - Scraping Page {page}...", end=" ")
            
            try:
                try:
                    driver.get(target_url)
                except TimeoutException:
                    driver.execute_script("window.stop();")

                # Scroll Logic
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                
                # Extract
                products = extract_data(driver, cat_name)
                
                # --- STOPPING CONDITION ---
                if len(products) == 0:
                    print("-> No products found. End of Category.")
                    break # Exit the loop for this category
                
                all_products.extend(products)
                print(f"-> Found {len(products)} items.")
                
                page += 1 # Go to next page

            except (NoSuchWindowException, WebDriverException) as e:
                print(f"\n    !!! Browser Error. Restarting driver...")
                try: driver.quit()
                except: pass
                driver = setup_driver()
                continue # Try the same page again or move on? Usually better to move on to avoid loops.
                
            time.sleep(1) 

    driver.quit()

    print("-------------------")
    if all_products:
        df = pd.DataFrame(all_products)
        df.to_csv("banggood_full_data.csv", index=False)
        print(f"Success! Total Scraped: {len(all_products)} items.")
        print("File saved: 'banggood_full_data.csv'")
    else:
        print("No data scraped. Check links.")

if __name__ == "__main__":
    main()

Starting Scraper (Unlimited Pages Mode)...
------------------------------------------
Processing Category: Test & Measuring Module
-> Found 30 items.1... 
  - Scraping Page 2... -> Found 60 items.
  - Scraping Page 3... -> Found 60 items.
  - Scraping Page 4... -> Found 18 items.
  - Scraping Page 5... -> No products found. End of Category.
Processing Category: USB Hubs
-> Found 60 items.1... 
  - Scraping Page 2... -> Found 16 items.
  - Scraping Page 3... -> No products found. End of Category.
Processing Category: Smart Watch
-> Found 30 items.1... 
  - Scraping Page 2... -> Found 60 items.
  - Scraping Page 3... -> Found 19 items.
  - Scraping Page 4... -> No products found. End of Category.
Processing Category: Handheld Vacuum Cleaners
-> Found 25 items.1... 
  - Scraping Page 2... -> No products found. End of Category.
Processing Category: Router
-> Found 30 items.1... 
  - Scraping Page 2... -> Found 30 items.
  - Scraping Page 3... -> Found 60 items.
  - Scraping Page 4... -> Fo

In [10]:
# banggood_scraper.py
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import json
from urllib.parse import urljoin, urlparse
import os

# ----------------------------- CONFIG -----------------------------
# Be respectful! Don't hammer the server
DELAY_BETWEEN_REQUESTS = random.uniform(2, 5)  # Random delay in seconds
MAX_PAGES_PER_CATEGORY = 3  # Set to higher if needed (e.g., 10)

# Use realistic headers
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
}

# Optional: Add proxies if you get blocked
PROXIES = {
    # "http": "http://your_proxy:port",
    # "https": "http://your_proxy:port",
}

# 5 Popular Banggood categories (as of 2025)
CATEGORIES = [
    {
        "name": "RC Drones & Accessories",
        "url": "https://www.banggood.com/Toys-Hobbies-and-Robot/RC-Drones-and-Accessories-c-12179.html"
    },
    {
        "name": "Smartphones",
        "url": "https://www.banggood.com/Consumer-Electronics/Mobile-Phones-c-10464.html"
    },
    {
        "name": "Laptops & Tablets",
        "url": "https://www.banggood.com/Computers-and-Office/Laptops-and-Tablets-c-10784.html"
    },
    {
        "name": "Tools & Home Improvement",
        "url": "https://www.banggood.com/Tools-and-Home-Improvement-c-12186.html"
    },
    {
        "name": "LED Lights & Flashlights",
        "url": "https://www.banggood.com/Lights-and-Lighting/LED-Lights-and-Flashlights-c-12190.html"
    }
]

# Output file
OUTPUT_CSV = "banggood_products.csv"
# ----------------------------------------------------------------

def get_soup(url):
    """Fetch page and return BeautifulSoup object"""
    try:
        time.sleep(DELAY_BETWEEN_REQUESTS)
        response = requests.get(url, headers=HEADERS, proxies=PROXIES, timeout=20)
        
        if response.status_code == 503 or "cloudflare" in response.text.lower():
            print(f"[!] Cloudflare protection detected on {url}")
            return None
            
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"[!] Request failed for {url}: {e}")
        return None

def extract_products_from_page(soup, category_name):
    """Extract product info from a single page"""
    products = []
    
    # Banggood product grid items (class changes sometimes, this works as of 2025)
    product_items = soup.select('.product-item') or soup.select('.good-item') or soup.select('li[data-product-id]')
    
    print(f"    Found {len(product_items)} products on this page")
    
    for item in product_items:
        try:
            # Product name
            title_tag = item.select_one('a.title, .title a, h3 a, .name a')
            name = title_tag.get_text(strip=True) if title_tag else "N/A"
            product_url = urljoin("https://www.banggood.com", title_tag['href']) if title_tag and title_tag.get('href') else "N/A"
            
            # Price
            price_tag = item.select_one('.price, .price-new, .good-price, [class*="price"]')
            price = price_tag.get_text(strip=True).replace('US$', '').strip() if price_tag else "N/A"
            
            # Rating
            rating_tag = item.select_one('.rating-num, .star-view span, .review-num')
            rating = rating_tag.get_text(strip=True) if rating_tag else "N/A"
            
            # Reviews count
            reviews_tag = item.select_one('.review-num, .review-count')
            reviews = reviews_tag.get_text(strip=True).strip('() ') if reviews_tag else "0"
            
            products.append({
                "category": category_name,
                "product_name": name,
                "price_usd": price,
                "rating": rating,
                "reviews": reviews,
                "product_url": product_url
            })
        except Exception as e:
            print(f"    Error parsing product: {e}")
            continue
    
    return products

def scrape_category(category):
    """Scrape multiple pages of a category"""
    all_products = []
    base_url = category["url"]
    
    print(f"\n[→] Scraping category: {category['name']}")
    
    for page in range(1, MAX_PAGES_PER_CATEGORY + 1):
        page_url = f"{base_url}?p={page}" if '?' in base_url else f"{base_url}?p={page}"
        print(f"  Scraping page {page}: {page_url}")
        
        soup = get_soup(page_url)
        if not soup:
            print(f"    Failed to retrieve page, possibly blocked.")
            break
            
        products = extract_products_from_page(soup, category["name"])
        if not products:
            print("    No products found – selector may have changed or page is empty.")
            break
            
        all_products.extend(products)
        print(f"    → Collected {len(products)} products (Total: {len(all_products)})")
        
        # Stop if no more products
        if len(products) < 20:  # Banggood usually shows 60 per page
            print("    Likely reached last page.")
            break
            
    print(f"Finished {category['name']}: {len(all_products)} products collected.")
    return all_products

def save_to_csv(all_data):
    """Save all scraped data to CSV"""
    keys = ["category", "product_name", "price_usd", "rating", "reviews", "product_url"]
    with open(OUTPUT_CSV, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(all_data)
    print(f"\nSaved {len(all_data)} products to {OUTPUT_CSV}")

def main():
    all_products = []
    
    for category in CATEGORIES:
        products = scrape_category(category)
        all_products.extend(products)
    
    if all_products:
        save_to_csv(all_products)
        print(f"\nScraping completed! Total products: {len(all_products)}")
    else:
        print("\nNo products scraped. Banggood may have updated structure or blocked requests.")

if __name__ == "__main__":
    main()


[→] Scraping category: RC Drones & Accessories
  Scraping page 1: https://www.banggood.com/Toys-Hobbies-and-Robot/RC-Drones-and-Accessories-c-12179.html?p=1
    Found 0 products on this page
    No products found – selector may have changed or page is empty.
Finished RC Drones & Accessories: 0 products collected.

[→] Scraping category: Smartphones
  Scraping page 1: https://www.banggood.com/Consumer-Electronics/Mobile-Phones-c-10464.html?p=1
    Found 0 products on this page
    No products found – selector may have changed or page is empty.
Finished Smartphones: 0 products collected.

[→] Scraping category: Laptops & Tablets
  Scraping page 1: https://www.banggood.com/Computers-and-Office/Laptops-and-Tablets-c-10784.html?p=1
    Found 0 products on this page
    No products found – selector may have changed or page is empty.
Finished Laptops & Tablets: 0 products collected.

[→] Scraping category: Tools & Home Improvement
  Scraping page 1: https://www.banggood.com/Tools-and-Home-Im

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

CATEGORIES = {
    "Chargers": "https://www.banggood.com/Wholesale-Chargers-ca-1055.html",
    "RC Cars": "https://www.banggood.com/Wholesale-RC-Car-ca-7008.html",
    "Multi Rotor Parts": "https://www.banggood.com/Wholesale-Multi-Rotor-Parts-ca-7014.html",
    "Woodworking Clamp": "https://www.banggood.com/Wholesale-Woodworking-Clamp-ca-18883.html",
    "Keyboard & Mouse": "https://www.banggood.com/Wholesale-Keyboards-and-Mouse-ca-5029.html"
}

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

PAGES_TO_SCRAPE = 2

def scrape_page(url, category_name):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"Failed to fetch {url}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    products_data = []

    # Try multiple container classes (similar to Selenium logic)
    containers = ["li.p-item", "div.p-wrap", "div.list-item", "li.gl-item", "div.product-item"]
    cards = []
    for c in containers:
        found = soup.select(c)
        if found:
            cards = found
            break

    if not cards:
        print("   > Warning: No product cards found (likely JS-loaded)")
        return []

    for card in cards:
        try:
            name_tag = card.select_one("a.title, a.p-title, .title a")
            name = name_tag.get_text(strip=True) if name_tag else "N/A"
            url = name_tag['href'] if name_tag and name_tag.has_attr('href') else "N/A"

            price_tag = card.select_one("span.price-box, .price, .product-price")
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            rating_tag = card.select_one("span.star")
            rating = rating_tag['data-score'] if rating_tag and rating_tag.has_attr('data-score') else "N/A"

            reviews_tag = card.select_one("a.review")
            reviews = reviews_tag.get_text(strip=True) if reviews_tag else "0 reviews"

            # Skip empty entries
            if name == "N/A" and price == "N/A":
                continue

            products_data.append({
                "Category": category_name,
                "Name": name,
                "Price": price,
                "Rating": rating,
                "Reviews": reviews,
                "URL": url
            })
        except Exception as e:
            print("Error parsing card:", e)
            continue

    return products_data

def main():
    all_products = []

    for cat_name, base_url in CATEGORIES.items():
        print(f"Processing Category: {cat_name}")
        for page in range(1, PAGES_TO_SCRAPE + 1):
            url = base_url if page == 1 else f"{base_url}?page={page}"
            print(f"  - Scraping Page {page}...")
            products = scrape_page(url, cat_name)
            print(f"    -> Found {len(products)} items.")
            all_products.extend(products)
            time.sleep(1)  # polite delay

    if all_products:
        df = pd.DataFrame(all_products)
        df.to_csv("banggood_products_bs.csv", index=False)
        print(f"Success! Scraped {len(all_products)} items.")
        print(df.head())
    else:
        print("No data scraped. Likely JS-loaded content.")

if __name__ == "__main__":
    main()


Processing Category: Chargers
  - Scraping Page 1...
    -> Found 0 items.
  - Scraping Page 2...
    -> Found 0 items.
Processing Category: RC Cars
  - Scraping Page 1...
    -> Found 0 items.
  - Scraping Page 2...
    -> Found 0 items.
Processing Category: Multi Rotor Parts
  - Scraping Page 1...
    -> Found 0 items.
  - Scraping Page 2...
    -> Found 0 items.
Processing Category: Woodworking Clamp
  - Scraping Page 1...
    -> Found 0 items.
  - Scraping Page 2...
    -> Found 0 items.
Processing Category: Keyboard & Mouse
  - Scraping Page 1...
    -> Found 0 items.
  - Scraping Page 2...
    -> Found 0 items.
No data scraped. Likely JS-loaded content.


In [ ]:
import pandas as pd

# 1. Load Data
df = pd.read_csv("banggood_products.csv")

# 2. Clean Price (Overwrite the 'Price' column directly)
# Remove messy symbols: Â, £, $, US
df["Price"] = df["Price"].astype(str).str.replace("Â", "").str.replace("£", "").str.replace("$", "").str.replace("US", "")
df["Price"] = pd.to_numeric(df["Price"], errors='coerce')

# 3. Clean Reviews (Overwrite the 'Reviews' column directly)
# Remove " reviews" and brackets
df["Reviews"] = df["Reviews"].astype(str).str.replace(" reviews", "").str.replace("(", "").str.replace(")", "")
df["Reviews"] = pd.to_numeric(df["Reviews"], errors='coerce').fillna(0)



# --- NEW STEP: DELETE THE ORIGINAL RATING COLUMN ---
if "Rating" in df.columns:
    df = df.drop(columns=['Rating'])
    print("Rating column deleted successfully.")
# --------------------------------------------------

# 5. Create 2 New Features (Using the now clean columns)
df["Total_Revenue"] = df["Price"] * df["Reviews"]
df["Is_Best_Seller"] = df["Reviews"] > 50 

# 6. Save (Now the file won't have any messy columns)
df.to_csv("banggood_products_cleaned.csv", index=False)
print("Done! Original columns are now clean.")
print(df.head())

In [13]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchWindowException
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIGURATION ---
CATEGORIES = {
    "Chargers": "https://www.banggood.com/Wholesale-Chargers-ca-1055.html",
    "RC Cars": "https://www.banggood.com/Wholesale-RC-Car-ca-7008.html",
    "Multi Rotor Parts": "https://www.banggood.com/Wholesale-Multi-Rotor-Parts-ca-7014.html",
    "Woodworking Clamp": "https://www.banggood.com/Wholesale-Woodworking-Clamp-ca-18883.html",
    "Keyboard & Mouse": "https://www.banggood.com/Wholesale-Keyboards-and-Mouse-ca-5029.html"
}
PAGES_TO_SCRAPE = 2 

def setup_driver():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    )
    options.page_load_strategy = 'eager' 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.set_page_load_timeout(30)
    return driver

def extract_data(driver, category_name):
    products_data = []
    possible_containers = [
        "li.p-item", "div.p-wrap", "div.list-item", "li.gl-item", "div.product-item", "ul.p-list-items li"
    ]
    cards = []
    for container in possible_containers:
        try:
            cards = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, container))
            )
            if len(cards) > 0:
                break
        except TimeoutException:
            continue 

    if not cards:
        return []

    for card in cards:
        name, price, url, rating, reviews, stock_status = "N/A", "N/A", "N/A", "N/A", "0 reviews", "Unknown"
        # 1. NAME
        try:
            name = card.find_element(By.CSS_SELECTOR, "a.title, a.p-title").text.strip()
        except:
            pass
        # 2. PRICE
        try:
            price = card.find_element(By.CSS_SELECTOR, "span.price-box, .price, .product-price").text.strip()
        except:
            pass
        # 3. URL
        try:
            url = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        except:
            pass
        # 4. REVIEWS
        try:
            reviews = card.find_element(By.CSS_SELECTOR, "a.review").text.strip()
        except:
            pass
        # 5. RATING
        try:
            rating_element = card.find_element(By.CSS_SELECTOR, "span.star, div.star, i.star")
            rating = rating_element.get_attribute("data-score") or rating_element.get_attribute("title")
            if not rating or not re.match(r'^\d+(\.\d+)?$', str(rating).strip()):
                rating = "N/A"
        except:
            rating = "N/A"
        # 6. STOCK AVAILABILITY
        try:
            stock_element = card.find_element(By.CSS_SELECTOR, ".stock, .availability")
            stock_text = stock_element.text.lower()
            if "in stock" in stock_text:
                stock_status = "In Stock"
            elif "out of stock" in stock_text or "sold out" in stock_text:
                stock_status = "Out of Stock"
            else:
                stock_status = stock_text
        except:
            stock_status = "Unknown"
        # Only append if name or price is present
        if name != "N/A" or price != "N/A":
            products_data.append({
                "Category": category_name,
                "Name": name,
                "Price": price,
                "Rating": rating,
                "Reviews": reviews,
                "Stock": stock_status,
                "URL": url
            })
    return products_data

def main():
    driver = setup_driver()
    all_products = []
    print("Starting Headless Scraper (with Rating and Stock)...")
    print("-----------------------------------------------")
    for cat_name, base_url in CATEGORIES.items():
        print(f"Processing Category: {cat_name}")
        for page in range(1, PAGES_TO_SCRAPE + 1):
            if page == 1: target_url = base_url
            else: target_url = f"{base_url}?page={page}"
            print(f"  - Scraping Page {page}...", end=" ")
            try:
                try:
                    driver.get(target_url)
                except TimeoutException:
                    driver.execute_script("window.stop();")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 2);")
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)
                products = extract_data(driver, cat_name)
                all_products.extend(products)
                print(f"-> Found {len(products)} items.")
            except (NoSuchWindowException, WebDriverException):
                print(f"\n    !!! Browser Error. Restarting driver...")
                try:
                    driver.quit()
                except:
                    pass
                driver = setup_driver()
                continue
            time.sleep(1) 
    driver.quit()
    print("-------------------")
    if all_products:
        df = pd.DataFrame(all_products)
        df.to_csv("banggood_products.csv", index=False)
        print(f"Success! Scraped {len(all_products)} items.")
        print("Data saved to 'banggood_products.csv'")
        print(df.head())
    else:
        print("No data scraped.")

if __name__ == "__main__":
    main()


Starting Headless Scraper (with Rating and Stock)...
-----------------------------------------------
Processing Category: Chargers
-> Found 0 items. 1... 
  - Scraping Page 2... -> Found 0 items.
Processing Category: RC Cars
  - Scraping Page 1... 

KeyboardInterrupt: 

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set styling for the plot
sns.set_theme(style="whitegrid")

# Load Data (Using the cleaned file)
try:
    df = pd.read_csv("banggood_products_cleaned.csv")
    print("--- Graph 5: Stock Availability Analysis (Count Plot) ---")
except FileNotFoundError:
    print("Error: 'banggood_products_cleaned.csv' file not found. Please ensure the cleaning script has been run.")
    exit()
    
# Check if the Availability_Marker column exists and has been created by the cleaning script
if 'Availability_Marker' not in df.columns:
    print("Error: 'Availability_Marker' column is missing. Please run the updated cleaning script first.")
    exit()

# GRAPH 5: Stock Availability Analysis (Count Plot)
# This graph shows the proportion of products identified by the Availability_Marker.
plt.figure(figsize=(10, 6))

# Define colors for better contrast
palette_colors = {'Standard Stock': '#007ACC', 'Pre-Order/New Stock': '#FFC300', 'Out of Stock': '#E35E5E'}

# Ensure the count plot is created correctly
marker_counts = df['Availability_Marker'].value_counts()
ordered_markers = marker_counts.index.tolist()

ax = sns.countplot(
    data=df, 
    y='Availability_Marker', # y-axis is the marker
    order=ordered_markers, # Order by count
    palette=[palette_colors.get(m, '#999999') for m in ordered_markers] # Get colors based on marker name
)

plt.title('Stock Availability vs. New/Pre-Order Products (Inferred from Name)', fontsize=16)
plt.xlabel('Number of Products', fontsize=12)
plt.ylabel('Availability Marker', fontsize=12)

# Add percentage labels to the bars
total = len(df)
for p in ax.patches:
    percentage = '{:.1f}%'.format(100 * p.get_width() / total)
    x = p.get_width()
    y = p.get_y() + p.get_height() / 2
    # Place text label to the right of the bar
    ax.annotate(percentage, (x, y), ha='left', va='center', fontsize=10, xytext=(5, 0), textcoords='offset points')

plt.show()

--- Graph 5: Stock Availability Analysis (Count Plot) ---
Error: 'Availability_Marker' column is missing. Please run the updated cleaning script first.


KeyError: 'Availability_Marker'

<Figure size 1000x600 with 0 Axes>